In [1]:
import json
from glob import glob
import fnmatch
from multiprocessing import Pool
from pathlib import Path
import os, itertools
from typing import *
from tqdm.notebook import tqdm
from importlib import reload
from pickle import dumps, loads, dump, load

from stexls import *

In [2]:
%rm /tmp/linker.bin

rm: cannot remove '/tmp/linker.bin': No such file or directory


In [3]:
root = Path.home()/'MathHub'

In [4]:
try:
    with open('/tmp/linker.bin', 'rb') as fd:
        linker = load(fd)
except Exception as e:
    print(e)
    linker = Linker(root=root, file_pattern='**/*.tex')
    print(linker.update(progress=tqdm, use_multiprocessing=True))
    with open('/tmp/linker.bin', 'w+b') as fd:
        dump(linker, fd)


[Errno 2] No such file or directory: '/tmp/linker.bin'



{}


In [5]:
%ls -ahl /tmp/linker.bin

-rw-rw-r-- 1 marian marian 157M Apr 11 07:25 /tmp/linker.bin


In [6]:
for o, link in linker.links.items():
    if link.errors:
        for loc, errs in link.errors.items():
            for err in errs:
                print(loc.format_link(True, root), err)
            print()
            print('-'*80)
            print()

MiKoMH/AI/source/rational-decisions/en/policy-iteration.tex:5:3 Module "sum/MODULE" previously imported at "/home/marian/MathHub/smglom/calculus/source/series.tex:3:3"
MiKoMH/AI/source/rational-decisions/en/policy-iteration.tex:5:3 Module "sum/MODULE" previously imported at "/home/marian/MathHub/MiKoMH/AI/source/probabilistic-reasoning/en/probabilistic-model.tex:7:1"
MiKoMH/AI/source/rational-decisions/en/policy-iteration.tex:5:3 Module "sum/MODULE" previously imported at "/home/marian/MathHub/MiKoMH/AI/source/probabilistic-reasoning/en/Markov-processes.tex:5:3"
MiKoMH/AI/source/rational-decisions/en/policy-iteration.tex:5:3 Module "sum/MODULE" previously imported at "/home/marian/MathHub/MiKoMH/AI/source/rational-decisions/en/utility-states.tex:3:3"
MiKoMH/AI/source/rational-decisions/en/policy-iteration.tex:5:3 Module "sum/MODULE" previously imported at "/home/marian/MathHub/MiKoMH/AI/source/rational-decisions/en/expected-utility.tex:3:3"
MiKoMH/AI/source/rational-decisions/en/policy

MiKoMH/talks/source/formalcad/en/flange-ontology.tex:14:14 Undefined symbol: "iso/MODULE"

--------------------------------------------------------------------------------

MiKoMH/talks/source/formalcad/en/flange-ontology.tex:14:7 Undefined symbol: "iso.ISO/SYMBOL"

--------------------------------------------------------------------------------

MiKoMH/IWGS/source/webapps/en/bottle-stpl.tex:27:27 Undefined symbol: "unicode-ucs.unicode-string/SYMBOL"

--------------------------------------------------------------------------------

MiKoMH/TDM/source/vc/en/git-branchfork.tex:2:3 Module "git-working/MODULE" previously imported at "/home/marian/MathHub/MiKoMH/IWGS/source/vci/en/gitlabhub.tex:2:3"

--------------------------------------------------------------------------------

MiKoMH/TDM/source/vc/en/git-branchfork.tex:3:3 Module "branching/MODULE" previously imported at "/home/marian/MathHub/MiKoMH/TDM/source/vc/en/git.tex:3:1"

----------------------------------------------------------

smglom/functional-analysis/source/topvr-Hausdorff.en.tex:4:20 Undefined symbol: "topological-vectorspace/MODULE"

--------------------------------------------------------------------------------

smglom/functional-analysis/source/topvr-Hausdorff.en.tex:4:11 Undefined symbol: "topological-vectorspace.topological-vector-space/SYMBOL"

--------------------------------------------------------------------------------

smglom/functional-analysis/source/topvr-Hausdorff.en.tex:5:13 Undefined symbol: "Hausdorff-topology/MODULE"

--------------------------------------------------------------------------------

smglom/functional-analysis/source/topvr-Hausdorff.en.tex:5:5 Undefined symbol: "Hausdorff-topology.Hausdorff-space/SYMBOL"

--------------------------------------------------------------------------------

MiKoMH/CompLog/source/kr/en/role-composition.tex:2:1 Module "alc-semantics/MODULE" previously imported at "/home/marian/MathHub/MiKoMH/CompLog/source/kr/en/alc-abox.tex:2:1"

-----------

smglom/meta-inf/lib/pre.ar.tex:1:1 Broken parser stack: [[Node "\documentclass[mh]{smglom}
\usepackage{arabtex}
{\makeatletter\long\gdef\@gobble#1{}}
\usepackage[utf8]{inputenc}
\usepackage[LFE,LAE]{fontenc}
\usepackage[arabic,english]{babel}
% this is a terrible hack that may introduce unwanted whitespace
\def\ignorespacesandparsafterend{}
\input{localpaths}
\libinput{preamble}\selectlanguage{english}
\begin{document}"], [Environment name=[Node "{document}"]]]

--------------------------------------------------------------------------------

MiKoMH/IWGS/source/digdocs/en/trees-cs.tex:20:27 Question mark syntax "?<symbol>" syntax not allowed in non-mtrefi environments.

--------------------------------------------------------------------------------

MiKoMH/IWGS/source/digdocs/en/trees-cs.tex:20:60 Question mark syntax "?<symbol>" syntax not allowed in non-mtrefi environments.

--------------------------------------------------------------------------------

MiKoMH/IWGS/source/digdocs/

smglom/functional-analysis/source/Heine-Borel.de.tex:7:3 Undefined symbol: "topological-vectorspace.base-set/SYMBOL"

--------------------------------------------------------------------------------

MiKoMH/talks/source/omdoc/en/education.tex:2:1 Module "omdoc-nutshell/MODULE" previously imported at "/home/marian/MathHub/MiKoMH/talks/source/mbase/en/nutshell.tex:3:1"

--------------------------------------------------------------------------------

MiKoMH/AI/source/search/en/simulated-annealing.tex:3:1 Module "infinity/MODULE" previously imported at "/home/marian/MathHub/MiKoMH/GenCS/source/dmath/en/sets-sizes.tex:4:1"
MiKoMH/AI/source/search/en/simulated-annealing.tex:3:1 Module "infinity/MODULE" previously imported at "/home/marian/MathHub/MiKoMH/AI/source/search/en/search-strategy.tex:3:1"

--------------------------------------------------------------------------------

MiKoMH/AI/source/ml/en/MLP-learning.tex:4:3 Module "binary-logarithm/MODULE" previously imported at "/home/marian

MiKoMH/GenCS/source/www/en/www-overview.tex:2:1 Module "url-urn/MODULE" previously imported at "/home/marian/MathHub/MiKoMH/GenCS/source/www/en/webserver.tex:2:1"

--------------------------------------------------------------------------------

smglom/calculus/source/metric-continuous-uniformly-continuous.en.tex:5:12 Undefined symbol: "metric-space/MODULE"

--------------------------------------------------------------------------------

smglom/calculus/source/metric-continuous-uniformly-continuous.en.tex:5:3 Undefined symbol: "metric-space.metric-space/SYMBOL"

--------------------------------------------------------------------------------

smglom/calculus/source/metric-continuous-uniformly-continuous.en.tex:6:11 Undefined symbol: "uniform-continuity/MODULE"

--------------------------------------------------------------------------------

smglom/calculus/source/metric-continuous-uniformly-continuous.en.tex:6:3 Undefined symbol: "uniform-continuity.uniformly-continuous/SYMBOL"

----

MiKoMH/GenICT/source/programming/en/files.tex:16:7 Undefined symbol: "hardware-software-programming.storage/SYMBOL"

--------------------------------------------------------------------------------

MiKoMH/GenICT/source/programming/en/files.tex:47:23 Undefined symbol: "files.stream/SYMBOL"

--------------------------------------------------------------------------------

smglom/arithmetics/source/commutativemonoid-nattimes.de.tex:4:12 Undefined symbol: "commutativemonoid/MODULE"

--------------------------------------------------------------------------------

smglom/arithmetics/source/commutativemonoid-nattimes.de.tex:4:3 Undefined symbol: "commutativemonoid.commutative-monoid/SYMBOL"

--------------------------------------------------------------------------------

MiKoMH/AI/source/nlp/en/IR-VSmodel.tex:12:36 Undefined symbol: "multiset.bag/SYMBOL"

--------------------------------------------------------------------------------

MiKoMH/AI/source/nlp/en/IR-VSmodel.tex:26:11 Undefined

MiKoMH/GenCS/source/dmath/en/indset.tex:55:3 Multiple imports of module "peano-axioms/MODULE", first imported in line 28, column 6.

--------------------------------------------------------------------------------

MiKoMH/GenCS/source/dmath/en/indset.tex:37:7 Multiple imports of module "unary-term/MODULE", first imported in line 29, column 6.

--------------------------------------------------------------------------------

MiKoMH/GenCS/source/dmath/en/indset.tex:56:3 Multiple imports of module "unary-term/MODULE", first imported in line 29, column 6.

--------------------------------------------------------------------------------

MiKoMH/GenCS/source/dmath/en/indset.tex:2:1 Module "structure/MODULE" previously imported at "/home/marian/MathHub/smglom/sets/source/preorder.tex:2:3"

--------------------------------------------------------------------------------

MiKoMH/GenCS/source/dmath/en/indset.tex:19:41 Undefined symbol: "function-properties/MODULE"

------------------------------

smglom/topology/source/fundamental-group.en.tex:7:5 Undefined symbol: "based-loop.loop/SYMBOL"

--------------------------------------------------------------------------------

MiKoMH/ComSem/source/dlc/en/grammar.tex:3:3 Module "BNF/MODULE" previously imported at "/home/marian/MathHub/MiKoMH/ComSem/source/drt/en/DRT-syntax.tex:4:1"

--------------------------------------------------------------------------------

smglom/functional-analysis/source/norm-metric.en.tex:2:56 Undefined symbol: "norm/MODULE"

--------------------------------------------------------------------------------

smglom/functional-analysis/source/norm-metric.en.tex:2:47 Undefined symbol: "norm.normed-vector-space/SYMBOL"

--------------------------------------------------------------------------------

smglom/functional-analysis/source/norm-metric.en.tex:3:28 Undefined symbol: "norm-induced-metric/MODULE"

--------------------------------------------------------------------------------

smglom/functional-analysis/s

--------------------------------------------------------------------------------

MiKoMH/CompLog/source/kr/en/alc-semantics.tex:5:1 Module "sets-operations/MODULE" previously imported at "/home/marian/MathHub/MiKoMH/CompLog/source/pl1/en/pl1-signature.tex:2:1"
MiKoMH/CompLog/source/kr/en/alc-semantics.tex:5:1 Module "sets-operations/MODULE" previously imported at "/home/marian/MathHub/MiKoMH/CompLog/source/pl1/en/pl1-syntax.tex:3:1"
MiKoMH/CompLog/source/kr/en/alc-semantics.tex:5:1 Module "sets-operations/MODULE" previously imported at "/home/marian/MathHub/MiKoMH/GenCS/source/dmath/en/relations.tex:2:1"
MiKoMH/CompLog/source/kr/en/alc-semantics.tex:5:1 Module "sets-operations/MODULE" previously imported at "/home/marian/MathHub/MiKoMH/GenCS/source/logic/en/logical-system.tex:2:1"
MiKoMH/CompLog/source/kr/en/alc-semantics.tex:5:1 Module "sets-operations/MODULE" previously imported at "/home/marian/MathHub/MiKoMH/GenCS/source/logic/en/substitution-extension.tex:4:1"
MiKoMH/CompLog/sourc

In [7]:
proglang_path = Path('/home/marian/MathHub/MiKoMH/KWARC/source/missing/en/proglang.tex')
for proglang_object in linker.objects[proglang_path]:
    proglang_link = linker.links[proglang_object]
proglang_order = linker.build_orders[proglang_object]

In [8]:
microML_path = Path('/home/marian/MathHub/MiKoMH/GenCS/source/machines/en/microML.tex')
for microML_object in linker.objects[microML_path]:
    microML_link = linker.links[microML_object]
microML_order = linker.build_orders[microML_object]

In [9]:
p = latex.parser.LatexParser(Path('/home/marian/MathHub/MiKoMH/IWGS/source/progintro/en/pythonanywhere.tex'))
p.parse()
p.root.children[-1].children[-1].children[1].children[2].children[2]

[Environment name=[Token begin=2774 end=2784 "\inlinedef"]]

In [10]:
for o in linker.objects[Path('/home/marian/MathHub/MiKoMH/IWGS/source/progintro/en/pythonanywhere.tex')]:
    print(o.format())

Contains files:
	/home/marian/MathHub/MiKoMH/IWGS/source/progintro/en/pythonanywhere.tex

Depends on:
	MiKoMH/IWGS/source/progintro/en/pythonanywhere.tex:2:1:public MODULE python-nutshell/MODULE from "/home/marian/MathHub/MiKoMH/GenICT/source/python/en/nutshell.tex"
	MiKoMH/IWGS/source/progintro/en/pythonanywhere.tex:3:1:public MODULE operatingsystems/MODULE from "/home/marian/MathHub/MiKoMH/KWARC/source/missing/en/operatingsystems.tex"
	MiKoMH/IWGS/source/progintro/en/pythonanywhere.tex:4:1:public MODULE pseudocode/MODULE from "/home/marian/MathHub/MiKoMH/KWARC/source/missing/en/pseudocode.tex"

Symbols:
	MiKoMH/IWGS/source/progintro/en/pythonanywhere.tex:1:19:public pythonanywhere/MODULE
	MiKoMH/IWGS/source/progintro/en/pythonanywhere.tex:5:1:public pythonanywhere.pythonAnywhere/SYMBOL

References:
	MiKoMH/IWGS/source/progintro/en/pythonanywhere.tex:2:1:python-nutshell/MODULE
	MiKoMH/IWGS/source/progintro/en/pythonanywhere.tex:3:1:operatingsystems/MODULE
	MiKoMH/IWGS/source/progintro

In [11]:
for o in linker.objects[Path('/home/marian/MathHub/MiKoMH/GenICT/source/python/en/methods.tex')]:
    print(o.format())

Contains files:
	/home/marian/MathHub/MiKoMH/GenICT/source/python/en/methods.tex

Depends on:
	MiKoMH/GenICT/source/python/en/methods.tex:2:1:public MODULE python-functions/MODULE from "/home/marian/MathHub/MiKoMH/GenICT/source/python/en/functions.tex"
	MiKoMH/GenICT/source/python/en/methods.tex:3:1:public MODULE python-datatypes/MODULE from "/home/marian/MathHub/MiKoMH/GenICT/source/python/en/datatypes.tex"
	MiKoMH/GenICT/source/python/en/methods.tex:4:1:public MODULE pseudocode/MODULE from "/home/marian/MathHub/MiKoMH/KWARC/source/missing/en/pseudocode.tex"
	MiKoMH/GenICT/source/python/en/methods.tex:5:1:public MODSIG oop-class/MODULE from "/home/marian/MathHub/smglom/computing/source/oop-class.tex"

Symbols:
	MiKoMH/GenICT/source/python/en/methods.tex:1:19:public python-methods/MODULE
	MiKoMH/GenICT/source/python/en/methods.tex:32:7:public python-methods.object/SYMBOL
	MiKoMH/GenICT/source/python/en/methods.tex:33:20:public python-methods.dot-notation/SYMBOL
	MiKoMH/GenICT/source/py

In [14]:
with open(Path.home() / 'Downloads' / 'full-dump.txt', 'w') as fd:
    for l in linker.links.values():
        fd.write(l.format())

KeyboardInterrupt: 

In [12]:
linker.info('/home/marian/MathHub/MiKoMH/GenICT/source/python/en/methods.tex')

Contains files:
	/home/marian/MathHub/smglom/computing/source/data.tex
	/home/marian/MathHub/smglom/mv/source/piecewise.tex
	/home/marian/MathHub/smglom/computing/source/coding.tex
	/home/marian/MathHub/smglom/sets/source/converse-relation.tex
	/home/marian/MathHub/smglom/sets/source/reflexive.tex
	/home/marian/MathHub/smglom/computing/source/programming-paradigm.tex
	/home/marian/MathHub/smglom/computing/source/memory.tex
	/home/marian/MathHub/smglom/sets/source/identity-function.tex
	/home/marian/MathHub/MiKoMH/GenICT/source/python/en/datatypes.tex
	/home/marian/MathHub/smglom/arithmetics/source/numbers.tex
	/home/marian/MathHub/smglom/sets/source/set.tex
	/home/marian/MathHub/smglom/arithmetics/source/natarith.tex
	/home/marian/MathHub/MiKoMH/GenICT/source/python/en/methods.tex
	/home/marian/MathHub/smglom/computing/source/medium.tex
	/home/marian/MathHub/smglom/sets/source/reflexive-extension.tex
	/home/marian/MathHub/smglom/computing/source/type.tex
	/home/marian/MathHub/smglom/co